In [1]:
from importlib import reload
import environments.ControlledRangeVariance
import MLE.MLE 

reload(environments.ControlledRangeVariance)
reload(MLE.MLE)

def batchtoonline(samples, seed=45):
    import numpy as np
    
    state = np.random.RandomState(seed)
    
    n = sum(c for c, w, r in samples)
    while n > 0:
        p = np.array([ c for c, w, r in samples ], dtype='float64') / n
        what = state.choice(len(samples), p=p)
        c = min(samples[what][0], 1)
        yield (c, samples[what][1], samples[what][2])
        samples[what] = (samples[what][0] - c, samples[what][1], samples[what][2]) 
        n -= c
    
env = environments.ControlledRangeVariance.ControlledRangeVariance(seed=45, wsupport=[0,2,1000], expwsq=100)
happrox = MLE.MLE.Online.HistApprox(wmin=0, wmax=1000, numbuckets=10)
onlineci = MLE.MLE.Online.CI(wmin=0, wmax=1000, rmin=0, rmax=1, alpha=0.05)
_, samples = env.sample(1 << 16)
from pprint import pformat
print(pformat(samples), flush=True)

for n, (c, w, r) in enumerate(batchtoonline(samples)):
    happrox.update(c, w, r)
    onlineci.update(happrox.iterator)
    
    if n & (n - 1) == 0 and n & 0xAAAAAAAA == 0:
        from pprint import pformat
        print(pformat(
                {
                    'n': n,
                    'onlineci': onlineci.getsoln(happrox.iterator),
                    'batchmle': MLE.MLE.estimate(happrox.iterator, wmin=0, wmax=1000), 
                    'batchci': MLE.MLE.asymptoticconfidenceinterval(happrox.iterator, wmin=0, wmax=1000)
                }), 
              flush=True)

[(35818, 0, 0), (5658, 2, 0), (24056, 2, 1), (2, 1000, 0), (2, 1000, 1)]
{'batchci': ((0, 1), (None, None)),
 'batchmle': (0.9999999999999999,
              {'betastar': 1.0000000000000002,
               'num': 1.0000000000000002,
               'qex': {0: 0.4709253351526474, 1000: 1.0028865081059628e-19},
               'qfunc': <function estimate.<locals>.<lambda> at 0x7fa7b1cf0730>,
               'vmax': 1.0,
               'vmin': 0.9999999999999999}),
 'n': 0,
 'onlineci': {'extra': {'beta': 0.7090212316407815,
                        'ci': False,
                        'n': 1.0000000000000002},
              'qfunc': <function Online.CI.getsoln.<locals>.<lambda> at 0x7fa783f8b048>}}
{'batchci': ((3.3230034397819513e-21, 4.701293974756043e+144),
             ({'betastar': -1.2207031253801468e-07,
               'gammastar': 9.999999999999999e-06,
               'kappastar': 3.323020615308456e-21,
               'qex': {(0, 0): 0.9989999999999999,
                       (1000, 0

{'batchci': ((0.6589297915712305, 0.8546360825826043),
             ({'betastar': -0.015439084270550716,
               'gammastar': 15.439094270550735,
               'kappastar': 0.015690326836304547,
               'qex': {(0, 0): 0.0, (1000, 0): 0.00015366266300262784},
               'qfunc': <function asymptoticconfidenceinterval.<locals>.<lambda> at 0x7fa7b1cf09d8>,
               'remone': 0.00015366066176181192,
               'remw': 0.15366266300462905,
               'sumone': 1.0000000020012407,
               'sumw': 0.9999999999979988},
              {'betastar': 0.9916675923054264,
               'gammastar': 8.332417694573424,
               'kappastar': 0.008262877471586726,
               'qex': {(0, 1): 0.0, (1000, 1): 0.00012117728894301755},
               'qfunc': <function asymptoticconfidenceinterval.<locals>.<lambda> at 0x7fa7b1cf08c8>,
               'remone': 0.00012115264827250449,
               'remw': 0.12117728896765823,
               'sumone': 1.00000